[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Classifier - The Linear Classifier

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 17/09/2022 | Royi Avital | First version                                                      |
|         |            |             |                                                                    |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/IntroductionMachineLearningSystemEngineers/ClassifierLinear.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import load_breast_cancer, make_circles
from sklearn.neighbors import KNeighborsClassifier

from scipy.spatial.distance import cdist

# Misc
import datetime
import os
from platform import python_version
import random
import warnings
import yaml

# Typing
from typing import Tuple

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, show

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

In [ ]:
# Configuration
%matplotlib inline

warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF = (8, 8)
ELM_SIZE_DEF = 50
CLASS_COLOR = ('b', 'r')


In [ ]:
# Fixel Algorithms Packages


In [ ]:
# Parameters

# Data Generation
numSamples = 1000
numSwaps = int(0.05 * numSamples)

# Ground Truth Classifier
paramA = -1
paramB = 0.3

# Data Visuzalization
figSize     = (8, 8)
elmSize     = 50
classColor0 = 'b'
classColor1 = 'r'

numGridPts = 250

In [ ]:
# Auxiliary Functions

def PlotBinaryClassData( mX: np.ndarray, vY: np.ndarray, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, elmSize: int = ELM_SIZE_DEF, classColor: Tuple[str, str] = CLASS_COLOR, axisTitle: str = None ) -> plt.Axes:

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)
    else:
        hF = hA.get_figure()
    
    vC, vN = np.unique(vY, return_counts = True)

    numClass = len(vC)
    if (len(vC) != 2):
        raise ValueError(f'The input data is not binary, the number of classes is: {numClass}')

    vIdx0 = vY == vC[0]
    vIdx1 = vY == vC[1] #<! Basically ~vIdx0

    hA.scatter(mX[vIdx0, 0], mX[vIdx0, 1], s = elmSize, color = classColor[0], edgecolor = 'k', label = f'$C_\u007b {vC[0]} \u007d$')
    hA.scatter(mX[vIdx1, 0], mX[vIdx1, 1], s = elmSize, color = classColor[1], edgecolor = 'k', label = f'$C_\u007b {vC[1]} \u007d$')
    hA.axvline(x = 0, color = 'k')
    hA.axhline(y = 0, color = 'k')
    hA.axis('equal')
    if axisTitle is not None:
        hA.set_title(axisTitle)
    hA.legend()
    
    return hA

## Generate / Load Data



In [ ]:
# Generate Data 
vL = np.array([paramA, paramB]) #<! The line is y = ax + b
mX = 4 * np.random.rand(numSamples, 2) - 2 #<! The box [-2, 2] x [-2, 2]
vY = paramA * mX[:, 0] + paramB < mX[:, 1] #<! Class 0: Below the curve, Class 1: Above the curve
vY[:numSwaps] = ~vY[:numSwaps]
vY = vY.astype(np.integer)

### Plot Data

In [ ]:
# Display the Data

hA = PlotBinaryClassData(mX, vY, axisTitle = 'Training Set')

## Linear Classifier

$$f_{\left( \boldsymbol{w} \right)} \left(\boldsymbol{x}\right)=\mathrm{sign}\left(\boldsymbol{w}^{T}\boldsymbol{x}-b\right)$$

Where $w$ are the parameters of the a linear plane.

In [ ]:
# Grid of the data support
vV       = np.linspace(-2, 2, numGridPts)
XX0, XX1 = np.meshgrid(vV, vV)
XX       = np.stack([XX0.flatten(), XX1.flatten()])

def PlotLinearClassifier(θ, b):
    vW = np.array([np.cos(np.deg2rad(θ)), np.sin(np.deg2rad(θ))])

    # vZ = (vW @ XX - vW[1] * b) > 0 #<! Moving from y = ax + b -> w1 x1 + w2 x2 - b = 0
    vZ = (vW @ XX - b) > 0
    ZZ = vZ.reshape(XX0.shape)
    
    # vHatY    = np.sign(vW @ mX.T - vW[1] * b) > 0 #<! Moving from y = ax + b -> w1 x1 + w2 x2 - b = 0
    vHatY    = np.sign(vW @ mX.T -b) > 0
    accuracy = np.mean(vY == vHatY)

    axisTitle = r'$f_{{w},b}\left({x}\right)={sign}\left({w}^{T}{x}-b\right)$' '\n' f'Accuracy = {accuracy:.2%}'

    hF, hA = plt.subplots(figsize = (8, 8))
    PlotBinaryClassData(mX, vY, hA = hA, axisTitle = axisTitle)
    v = np.array([-2, 2])
    hA.grid(True)
    # hA.plot(v, -(vW[0] / vW[1]) * v + b, color = 'k', lw = 3) #<! y = ax + b notation
    hA.plot(v, -(vW[0] / vW[1]) * v + (b / vW[1]), color = 'k', lw = 3) #<! y = ax + b notation
    hA.arrow(0, 0, vW[0], vW[1], color = 'orange', width = 0.05)
    hA.axvline(x = 0, color = 'k', lw = 1)
    hA.axhline(y = 0, color = 'k', lw = 1)
    hA.contourf(XX0, XX1, ZZ, colors = CLASS_COLOR, alpha = 0.2, levels = [-0.5, 0.5, 1.5], zorder = 0)
    
    hA.axis([-2, 2, -2, 2])
    hA.set_xlabel('$x_1$')
    hA.set_ylabel('$x_2$')
    
    plt.show()

In [ ]:
# Display the Geometry of the Classifier

θSlider = FloatSlider(min = 0, max = 360, step = 1, value = 30, layout = Layout(width = '30%'))
bSlider = FloatSlider(min = -2.5, max = 2.5, step = 0.1, value = -0.3, layout = Layout(width = '30%'))
interact(PlotLinearClassifier, θ = θSlider, b = bSlider)